In [18]:
import sympy
import control
import numpy as np
import matplotlib.pyplot as plt
import gradio
from sympy import E
from sympy import print_latex
from sympy.parsing.latex import parse_latex

s, z = sympy.var('s z')
t, T, K = sympy.var('t T K', positive=True)
k = sympy.var('k', nteger=True)


In [2]:
sys = 1 / (s**2 * (s + 1))
sys = sys.apart()
print_latex(sys)
sys

\frac{1}{s + 1} - \frac{1}{s} + \frac{1}{s^{2}}


1/(s + 1) - 1/s + s**(-2)

In [3]:
# def Z(expr):
#     return sympy.summation(expr * z**(-k), (k, 0, sympy.oo))


# def L(expr):
#     return sympy.laplace_transform(expr, t, s)


# def invL(expr):
#     return sympy.inverse_laplace_transform(expr, s, t)


# def sample(expr):
#     return expr.subs(t, k * T)


# Z(sample(invL(sys)))

In [4]:
# e := e^-T, p := z^-1
e, p = sympy.var('e p')

D = K * (15 - 5 * p)

G = (1 - p) * (1 / (1 - e * p) - 1 / (1 - p) + T * p / (1 - p)**2)
G = G.simplify()

Wo = D * G
Wo = Wo.simplify()
Wo_num = sympy.numer(Wo).as_poly(p)
Wo_den = sympy.denom(Wo).as_poly(p)

num = [c.collect(K) for c in Wo_num.all_coeffs()]
den = [c.collect(K) for c in Wo_den.all_coeffs()]
print_latex(num)
print_latex(den)
display(Wo)

Wc = Wo / (1 + Wo)
Wc = Wc.simplify()
Wc_num = sympy.numer(Wc).as_poly(p)
Wc_den = sympy.denom(Wc).as_poly(p)

num = [c.collect(K) for c in Wc_num.all_coeffs()]
den = [c.collect(K) for c in Wc_den.all_coeffs()]
print_latex(num)
print_latex(den)
display(Wc)

\left[ K \left(5 T e + 5 e - 5\right), \  K \left(- 15 T e - 5 T - 20 e + 20\right), \  K \left(15 T + 15 e - 15\right), \  0\right]
\left[ e, \  - e - 1, \  1\right]


5*K*p*(p - 3)*(T*e*p - T + e*p - e - p + 1)/(e*p**2 - e*p - p + 1)

\left[ K \left(5 T e + 5 e - 5\right), \  K \left(- 15 T e - 5 T - 20 e + 20\right), \  K \left(15 T + 15 e - 15\right), \  0\right]
\left[ K \left(5 T e + 5 e - 5\right), \  K \left(- 15 T e - 5 T - 20 e + 20\right) + e, \  K \left(15 T + 15 e - 15\right) - e - 1, \  1\right]


5*K*p*(p - 3)*(T*e*p - T + e*p - e - p + 1)/(5*K*p*(p - 3)*(T*e*p - T + e*p - e - p + 1) + e*p**2 - e*p - p + 1)

In [5]:
w = sympy.var('w')

Wcw = Wc.subs(p, (1 - w) / (1 + w)).simplify()
poly = sympy.denom(Wcw).as_poly(w)

coef = [c.collect(K) for c in poly.all_coeffs()]
print_latex(coef)
poly

\left[ K \left(- 20 T e - 20 T - 40 e + 40\right) + 2 e + 2, \  K \left(30 T e - 10 T + 20 e - 20\right) + 4, \  K \left(20 T + 20 e - 20\right) - 2 e + 2, \  K \left(- 10 T e + 10 T\right)\right]


Poly((-20*K*T*e - 20*K*T - 40*K*e + 40*K + 2*e + 2)*w**3 + (30*K*T*e - 10*K*T + 20*K*e - 20*K + 4)*w**2 + (20*K*T + 20*K*e - 20*K - 2*e + 2)*w - 10*K*T*e + 10*K*T, w, domain='ZZ[e,K,T]')

In [6]:
N = poly.degree(w)

A = sympy.zeros(N)
A[0] = [coef[0], coef[1]]
A[1] = [coef[2]]

for i in range(2, N):
    for j in range(0, N - i):
        A[i, j] = A[i - 2, j + 1] - A[i - 2, 0] * A[i - 1, j + 1] / A[i - 1, 0]
        A[i, j] = A[i, j].simplify()

A

\frac{K \left(T \left(e^{T} z - 1\right) + e^{T} \left(z - 1\right)^{2} - \left(z - 1\right) \left(e^{T} z - 1\right)\right) R{\left(s \right)}}{\left(z - 1\right) \left(e^{T} z - 1\right) \left(K{\left(\frac{T e^{T} z - T - e^{T} z + e^{T} + z - 1}{e^{T} z^{2} - e^{T} z - z + 1} \right)} + 1\right)}


K*(T*(e**T*z - 1) + e**T*(z - 1)**2 - (z - 1)*(e**T*z - 1))*R(s)/((z - 1)*(e**T*z - 1)*(K((T*e**T*z - T - e**T*z + e**T + z - 1)/(e**T*z**2 - e**T*z - z + 1)) + 1))

In [24]:
Ec = 1 / (1 + Wo)
Ec = Ec.simplify()
sys = Ec.subs(p, 1 / z).simplify()
sys_num = sympy.numer(sys).as_poly(z)
sys_den = sympy.denom(sys).as_poly(z)
sys

z*(-e - z**2 + z*(e + 1))/(5*K*(3*z - 1)*(T*e + e + z*(-T - e + 1) - 1) - e*z - z**3 + z**2*(e + 1))

In [39]:
def interface(k, t, p, v, a):
    dict = {K: k, T: t, e: E**-t}
    num = [float(a.subs(dict)) for a in sys_num.all_coeffs()]
    den = [float(a.subs(dict)) for a in sys_den.all_coeffs()]
    tf = control.tf(num, den, t)
    ts = np.arange(0, 20, t)
    us = p + v * ts + a / 2 * ts**2
    ts, ys = control.forced_response(tf, ts, us)
    fig = plt.figure(figsize=(8, 4))
    plt.stem(ts, ys)
    plt.close()
    return fig


gradio.Interface(interface, [
    gradio.Slider(0.1, 10, 1),
    gradio.Slider(0.1, 1, 0.2),
    gradio.Slider(0, 5, 1),
    gradio.Slider(0, 5, 2),
    gradio.Slider(0, 5, 0)
],
                 gradio.Plot(elem_id='plot'),
                 live=True).launch(width=1560)


/Users/hasined/.pyenv/versions/3.10.4/lib/python3.10/site-packages/gradio/blocks.py:155: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(
/Users/hasined/.pyenv/versions/3.10.4/lib/python3.10/site-packages/gradio/blocks.py:155: UserWarning: api_name predict already exists, using predict_2
  warnings.warn(
/Users/hasined/.pyenv/versions/3.10.4/lib/python3.10/site-packages/gradio/blocks.py:155: UserWarning: api_name predict already exists, using predict_3
  warnings.warn(
/Users/hasined/.pyenv/versions/3.10.4/lib/python3.10/site-packages/gradio/blocks.py:155: UserWarning: api_name predict already exists, using predict_4
  warnings.warn(


Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x16c8749d0>, 'http://127.0.0.1:7883/', None)